# Spin Lock

## Spin Lock

### include/linux/spinlock_types.h

```c
typedef struct spinlock {
	union {
		struct raw_spinlock rlock;

#ifdef CONFIG_DEBUG_LOCK_ALLOC
# define LOCK_PADSIZE (offsetof(struct raw_spinlock, dep_map))
		struct {
			u8 __padding[LOCK_PADSIZE];
			struct lockdep_map dep_map;
		};
#endif
	};
} spinlock_t;

```

```c
typedef struct raw_spinlock {
	arch_spinlock_t raw_lock;
#ifdef CONFIG_GENERIC_LOCKBREAK
	unsigned int break_lock;
#endif
#ifdef CONFIG_DEBUG_SPINLOCK
	unsigned int magic, owner_cpu;
	void *owner;
#endif
#ifdef CONFIG_DEBUG_LOCK_ALLOC
	struct lockdep_map dep_map;
#endif
} raw_spinlock_t;
```

1. `arch_spinlock_t` is a architecture-specific `spinlock`. On x86_64, it is `qspinlock`.

-------------------------

### some functions description

`spin_lock_init` - produces initialization of the given spinlock;
`spin_lock` - acquires given spinlock;
`spin_lock_bh` - disables software interrupts and acquire given spinlock;
`spin_lock_irqsave` and `spin_lock_irq` - disable interrupts on local processor, preserve/not preserve previous interrupt state in the flags and acquire given spinlock;
`spin_unlock` - releases given spinlock;
`spin_unlock_bh` - releases given spinlock and enables software interrupts;
`spin_is_locked` - returns the state of the given spinlock;

---------------------------------


## Queued Spin Lock

简单的spin lock实现以及存在的问题

```c
int lock(lock)
{
    while (test_and_set(lock) == 1)
        ;
    return 0;
}

int unlock(lock)
{
    lock=0;

    return lock;
}
```

存在的问题

1. 这种spin lock并不是先到先得的。多个线程在一个变量上spin，谁先获取具有随机性。并不是先到先得

2. 由于多个CPU线程均在同一个共享变量lock上自旋，而申请和释放锁的时候必须对lock进行修改，这将导致所有参与排队自旋锁操作的处理器的缓存变得无效。如果排队自旋锁竞争比较激烈的话，频繁的缓存同步操作会导致繁重的系统总线和内存的流量，从而大大降低了系统整体的性能。

-------------------------

### MCS Spin Lock

[MCS](http://www.cs.rochester.edu/~scott/papers/1991_TOCS_synch.pdf)

The basic idea of the MCS lock is that a thread spins on a local variable and each processor in the system has its own copy of this variable (see the previous paragraph). In other words this concept is built on top of the per-cpu variables concept in the Linux kernel.

When the first thread wants to acquire a lock, it registers itself in the queue. In other words it will be added to the special queue and will acquire lock, because it is free for now. When the second thread wants to acquire the same lock before the first thread release it, this thread adds its own copy of the lock variable into this queue. In this case the first thread will contain a next field which will point to the second thread. From this moment, the second thread will wait until the first thread release its lock and notify next thread about this event. The first thread will be deleted from the queue and the second thread will be owner of a lock.

pesudocode:

```c
void lock() 
{
    lock.next = NULL;
    ancestor = put_lock_to_queue_and_return_ancestor(queue, lock); 

    if(ancestor) {
        lock.is_lock = 1;
        ancestor.next = lock;
        while(lock.is_lock);
    }
}

void unlock()
{
    if(lock.next != NULL) {
        lock.next.is_locked = false;
    }
}

```

--------------------------------

### include/asm-generic/qspinlock_types.h

```c
typedef struct qspinlock {
	atomic_t	val;
} arch_spinlock_t;

/*
 * Bitfields in the atomic value:
 *
 * When NR_CPUS < 16K
 *  0- 7: locked byte
 *     8: pending
 *  9-15: not used
 * 16-17: tail index
 * 18-31: tail cpu (+1)
 *
 * When NR_CPUS >= 16K
 *  0- 7: locked byte
 *     8: pending
 *  9-10: tail index
 * 11-31: tail cpu (+1)
 */
```

--------------------